[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/tutorial/notebooks/extract_sans_reductionlog_metadata)

<img src='__docs/__all/notebook_rules.png' />

# Select Instrument

In [1]:
from __code.sans import extract
from __code.__all import custom_style
custom_style.style()

initializer = extract.Initializer()
initializer.select_instrument()

# Select your ReductionLog files and then the Metadata to extract 

In [2]:
working_dir = initializer.get_working_dir()
o_extract = extract.Extract(working_dir=working_dir,
                           instrument=initializer.get_instrument())
o_extract.select_reductionlog()

Label(value='Command + Click: to select more than 1 element in the right widget')

# Select output folder

In [5]:
o_extract.export()

OrderedDict([('/HFIR/CG2/s1mid_q__copy_reduction_log.hdf',
              OrderedDict([('reduction_information_sample_logs_main_wavelength',
                            array('[4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.75038654 4.74951919 4.75038654 4.74951919 4.75038654 4.74951919\n 4.750386

# DEBUG 

In [2]:
import h5py
import numpy as np

In [3]:
nexus = "/HFIR/CG2/s1mid_q__copy_reduction_log.hdf"

entry_path = []
entry_path.append(['reduction_information', 'sample_logs', 'main', 'wavelength'])
entry_path.append(['reduction_information', 'sample_logs', 'main', 'wavelength_spread'])
entry_path.append(['reduction_information', 'special_parameters', 'sample_transmission'])
entry_path.append(['reduction_information', 'special_parameters', 'beam_center'])

In [4]:
result = extract.Extract.get_entry_value(nexus_file_name=nexus,
                             list_entry_path=entry_path)

In [5]:
result.keys()

odict_keys(['reduction_information_sample_logs_main_wavelength', 'reduction_information_sample_logs_main_wavelength_spread', 'reduction_information_special_parameters_sample_transmission', 'reduction_information_special_parameters_beam_center'])

In [7]:
result['reduction_information_special_parameters_beam_center']

{'x': 0.004563300817405293, 'y': -0.023762345405625846}

In [99]:
with h5py.File(nexus, 'r') as nxs_path:
    for _item in entry_path:
        nxs_path = nxs_path.get(_item)
    
    if type(nxs_path) == h5py._hl.group.Group: 
        result = {}
        for _key in nxs_path.keys():
            value = nxs_path.get(_key)
            print(f"key:{_key} and value:{value}") 
            if type(value) == h5py._hl.dataset.Dataset: #sample_transmission, beam_center
                result[_key] = np.squeeze(value[()])       
            else:
                result[_key] = value
        print(result)
        
    if type(nxs_path) == h5py._hl.dataset.Dataset: # sample_logs -> wavelength
        print(np.array(nxs_path[()]))

        

[0.13235294 0.13235294]


In [1]:
from __code.sans import extract

IndentationError: expected an indented block (extract.py, line 240)

In [63]:
def get_entry_value(nexus, entry_path):

    with h5py.File(nexus, 'r') as nxs:
        try:
            nxs_path = nxs
            for _item in entry_path:
                nxs_path = nxs_path.get(_item)

        except AttributeError:
            raise AttributeError("Path specify in the HDF5 is wrong!")

        if nxs_path is None:
            return None

        if type(nxs_path) == h5py._hl.group.Group:
            result = {}
            for _key in nxs_path.keys():
                print("_key is {}".format(_key))
                value = nxs_path.get(_key)
                if type(value) == h5py._hl.dataset.Dataset:
                    result[_key] = value[()]
                else:
                    result[_key] = value

            return result

        if type(nxs_path) == h5py._hl.dataset.Dataset:
            print("dataset here")
            return np.array(nxs_path[()])

        return None

val = get_entry_value(nexus, entry_path)
if type(val) is dict:
    for _key in val.keys():
        print(f"val[{_key}] = {val[_key]}")
print(val)
print(type(val))

_key is error
_key is value
val[error] = [[0.00627902]]
val[value] = [[0.82357569]]
{'error': array([[0.00627902]]), 'value': array([[0.82357569]])}
<class 'dict'>
